In [ ]:
import xarray as xr
import s3sf as s3
import geopandas as gpd

In [ ]:


## Read in IMERGE (is this the best place for this?)

if (time_freq == "daily"):
    imerge = xr.open_dataset("s3://veda-data-store-staging/EIS/zarr/GEOS5_FWI_GPM_LATE_v5_Daily.zarr", engine="zarr")
    imerge.rio.write_crs("epsg:4326", inplace=True)
    imerge.rio.set_spatial_dims(x_dim = "lon", y_dim = "lat", inplace = True)
if (time_freq == "hourly"):
    imerge = xr.open_dataset("s3://veda-data-store-staging/EIS/zarr/FWI-GEOS-5-Hourly", engine="zarr")
    imerge = imerge["GEOS-5_FWI"]
    imerge.rio.write_crs("epsg:4326", inplace=True)
    imerge.rio.set_spatial_dims(x_dim = "lon", y_dim = "lat", inplace = True)


## Get imerge -- Need to do on whole area? How manke more computationally efficient?
final_perimeter = max(gdf[gdf.t == max(gdf.t)].geometry)
#print(final_perimeter.envelope.exterior.coords.xy)
lons = final_perimeter.envelope.exterior.coords.xy[0]
lats = final_perimeter.envelope.exterior.coords.xy[1]
img_clip = imerge.rio.clip_box(minx = min(lons), miny = min(lats), maxx=max(lons), maxy = max(lats), auto_expand= True)

if(add_anomolies):
    img_clip = fv.imerge_climate(img_clip ,clim = ["rank", "anomolie","rank_anomolie"], var = ["FWI"])

img_clip = img_clip.sel(time = slice(min(gdf.t), max(gdf.t))).mean(dim = ["lat", "lon"])
img_clip = img_clip.to_dataframe()
img_clip.drop(columns = "spatial_ref", inplace = True)
img_clip.dropna(inplace = True)

#gdf["fireID"] = id
gdf = gdf.rename(columns = {"t" : "time"})
gdf['time'] = gdf['time'].astype('datetime64[ns]')
full = pd.merge(gdf,img_clip, on = "time", how = "outer")
full = full.rename(columns = {"time": "t"}) ## Seems silly but I cant get the img_clip to convert it's name
